In [1]:
import csv
import os
import re
from collections import Counter
import pickle

import numpy as np
import xlrd
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText
from nltk import sent_tokenize, word_tokenize
from stanfordcorenlp import StanfordCoreNLP
from log import logger
from decorators import logging

class Document:
    def __init__(self, doc_id, title, description, permissions, tags=None, raw_sentences=None):
        self.id = doc_id
        self.title = title
        self.permissions = permissions
        self.description = description 
        self.tags = tags
        self.raw_sentences = raw_sentences

    def __str__(self):
        print(self.title)
        print(self.permissions)



class Permission:
    def __init__(self, permission_type, permission_phrase):
        self.ptype = permission_type
        self.pphrase = permission_phrase

    def __repr__(self):
        return "Permission({}, {})".format(self.ptype, " ".join(self.pphrase))

    def __eq__(self, other):
        if isinstance(other, Permission):
            return ((self.ptype == other.ptype))
        else:
            return False

    def __hash__(self):
        return hash(self.__repr__())


class Utils:
    CORE_NLP_DIR = r'/home/huseyin/LIB/stanford-corenlp-full-2018-10-05'
    
    @staticmethod
    @logging
    def preprocess(text):
        paragrahps = text.split("\n")
        sentences = []
        for p in paragrahps:
            for s in sent_tokenize(p):
                sentences.append(s)
        return sentences

    @staticmethod
    @logging
    def remove_hyperlinks(text):
        regex = r"((https?:\/\/)?[^\s]+\.[^\s]+)"
        text = re.sub(regex, '', text)
        return text

    @staticmethod
    def to_lower(w, lower):
        return w.lower() if lower else w
    
    @staticmethod
    @logging
    def vocab(file_path, file_type="csv", lower=True):
        nlp = StanfordCoreNLP(Utils.CORE_NLP_DIR)
        wordsCount = Counter()
        permissions = []
        distincts_permissions = set()
        if file_type == "csv":
            with open(file_path) as f:
                reader = csv.reader(f)
                next(reader) # skip header
                for row in reader:
                    text = row[1]
                    for sentence in Utils.preprocess(text):
                        sentence = Utils.remove_hyperlinks(sentence)
                        for w in nlp.word_tokenize(sentence):
                            wordsCount.update([Utils.to_lower(w, lower)])
                        for p in  row[2].strip().split(","):
                            ptype = Utils.to_lower(p, lower)
                            if ptype not in distincts_permissions:
                                pphrase = [Utils.to_lower(t, lower) for t in p.split("_")]
                                perm = Permission(ptype, pphrase)
                                permissions.append(perm)
                                distincts_permissions.add(ptype)
                            for token in p.split("_"):
                                wordsCount.update([Utils.to_lower(token, lower)])
        elif file_type == "excel":
            handtagged_permissions = ["READ_CALENDAR", "READ_CONTACTS", "RECORD_AUDIO"]
            loc = (file_path)
            wb = xlrd.open_workbook(loc) 
            sheet = wb.sheet_by_index(0)
            sharp_count = 0
            apk_title = ""
            for i in range(sheet.nrows):
                sentence = sheet.cell_value(i, 0)
                if sentence.startswith("##"):
                    sharp_count += 1
                    if sharp_count % 2 == 1:
                        apk_title = sentence.split("##")[1] 
                else:
                    if sharp_count != 0 and sharp_count % 2 == 0:
                        sentence = sentence.strip()
                        for w in nlp.word_tokenize(sentence):
                            wordsCount.update([Utils.to_lower(w, lower)])
                        for p in handtagged_permissions:
                            ptype = Utils.to_lower(p, lower)
                            if ptype not in distincts_permissions:
                                pphrase = [Utils.to_lower(t, lower) for t in p.split("_")]
                                perm = Permission(ptype, pphrase)
                                permissions.append(perm)
                                distincts_permissions.add(ptype)
                                for token in p.split("_"):
                                    wordsCount.update([Utils.to_lower(token, lower)])            
        else:
            raise Exception("Unsupported file type.")
        nlp.close()
        return wordsCount.keys(), {w: i for i, w in enumerate(list(wordsCount.keys()))}, permissions

    @staticmethod
    @logging
    def read_file(file_path, w2i, file_type="csv", lower=True):
        data = []
        doc_id = 0
        if file_type == "csv":
            with open(file_path) as f:
                reader = csv.reader(f)
                next(reader) # skip header
                for row in reader:
                    doc_id += 1
                    title = row[0]
                    description = row[1]
                    permissions = set()
                    for p in  row[2].strip().split(","):
                        ptype = Utils.to_lower(p, lower)
                        pphrase = [Utils.to_lower(t, lower) for t in p.split("_")]
                        perm = Permission(ptype, pphrase)
                        permissions.add(perm)

                    sentences = []
                    raw_sentences = []
                    for sentence in Utils.preprocess(description):
                        sentence = Utils.remove_hyperlinks(sentence)
                        sentences.append(sentence.strip())
                        raw_sentences.append(sentence.strip())
                        ###sentences.append([Utils.to_lower(w, lower) for w in word_tokenize(sentence)])           
                    yield Document(doc_id, title, sentences, permissions,raw_sentences=raw_sentences)
                    
        elif file_type == "excel":
            permission_title = file_path.split("/")[-1].split(".")[0]
            loc = (file_path)
            wb = xlrd.open_workbook(loc) 
            sheet = wb.sheet_by_index(0)
            sharp_count = 0
            title = ""
            permissions = set()
            sentences = []
            raw_sentences = []
            tags = []
            for i in range(sheet.nrows):
                sentence = sheet.cell_value(i, 0)
                if sentence.startswith("##"):
                    sharp_count += 1
                    if sharp_count % 2 == 1:
                        if doc_id > 0:
                            yield Document(doc_id, title, sentences, permissions, tags, raw_sentences)
                        
                        #Document init values
                        title = sentence.split("##")[1]
                        permissions = set()
                        sentences = []
                        raw_sentences = []
                        tags = []
                        doc_id += 1
                        
                        # Permissions for apk
                        ptype = Utils.to_lower(permission_title, lower)
                        pphrase = [Utils.to_lower(t, lower) for t in permission_title.split("_")]
                        perm = Permission(ptype, pphrase)
                        permissions.add(perm)
                else:
                    if sharp_count != 0 and sharp_count % 2 == 0:
                        sentences.append(sentence.strip())
                        raw_sentences.append(sentence.strip())
                        ###sentences.append([Utils.to_lower(w, lower) for w in word_tokenize(sentence.strip())]) 
                        tags.append(int(sheet.cell_value(i, 1)))
                        
            yield Document(doc_id, title, sentences, permissions, tags, raw_sentences)
            wb.release_resources()
            del wb
        else:
            raise Exception("Unsupported file type.")
    
    @staticmethod
    @logging
    def get_data(file_path, w2i, sequence_type="dependency", file_type="csv", window_size=2, lower=True):
        nlp = StanfordCoreNLP(Utils.CORE_NLP_DIR)
        if sequence_type == "raw":
            return Utils.read_file_raw(file_path, w2i, nlp, file_type, lower)
        elif sequence_type == "dependency":
            return Utils.read_file_dependency(file_path, w2i, nlp, file_type, lower)
        elif sequence_type == "windowed":
            return Utils.read_file_window(file_path, w2i, nlp, file_type, window_size, lower)
        else:
            nlp.close()
            raise Exception("Unknown sequence type")
        
    @staticmethod
    def read_file_raw(file_path, w2i, nlp, file_type="csv", lower=True):
        for doc in Utils.read_file(file_path, w2i, file_type, lower):
            doc.description = [[Utils.to_lower(w, lower) for w in nlp.word_tokenize(sentence)] for sentence in doc.description]
            yield doc
        
    @staticmethod
    def read_file_window(file_path, w2i, nlp, file_type="csv", window_size=2, lower=True):
        for doc in Utils.read_file(file_path, w2i, file_type, lower):
            doc.description = [[Utils.to_lower(w, lower) for w in nlp.word_tokenize(sentence)] for sentence in doc.description]
            doc.description = Utils.split_into_windows(doc.description, window_size)
            yield doc
            
    @staticmethod
    def read_file_dependency(file_path, w2i, nlp, file_type="csv", lower=True):
        for doc in Utils.read_file(file_path, w2i, file_type, lower):
            doc.description = Utils.split_into_dependencies(doc.description, nlp)
            yield doc
            
    @staticmethod
    def split_into_dependencies(sentences, nlp):
        splitted_sentences = []
        for sentence in sentences:
            tokens = nlp.word_tokenize(sentence)
            s = [[tokens[rel[1]-1], tokens[rel[2]-1]] for rel in nlp.dependency_parse(sentence) if rel[0] != 'ROOT']
            splitted_sentences.append(s)
        return splitted_sentences
    
    @staticmethod
    def split_into_windows(sentences, window_size=2):
        splitted_sentences = []
        for sentence in sentences:
            splitted_sentences.append([])
            if len(sentence) < window_size:
                splitted_sentences[-1].append(sentence)
            else:
                for start in range(len(sentence) - window_size + 1):
                    splitted_sentences[-1].append([sentence[i+start] for i in range(window_size)])
        return splitted_sentences
    
    @staticmethod     
    def load_embeddings_file(file_name, embedding_type, lower=True):
        if not os.path.isfile(file_name):
            raise Exception("{} does not exist".format(file_name))
            
        if embedding_type == "word2vec":
            model = KeyedVectors.load_word2vec_format(file_name, binary=True, unicode_errors="ignore")
            words = model.index2entity
        elif embedding_type == "fasttext":
            model = FastText.load_fasttext_format(file_name)
            words = [w for w in model.wv.vocab]
        elif embedding_type == "pickle":
            with open(file_name,'rb') as fp:
                model = pickle.load(fp)
                words = model.keys()
        else:
            raise Exception("Unknown Type")

        if lower:
            vectors = {word.lower(): model[word] for word in words}
        else:
            vectors = {word: model[word] for word in words}

        if "UNK" not in vectors:
            unk = np.mean([vectors[word] for word in vectors.keys()], axis=0)
            vectors["UNK"] = unk

        return vectors, len(vectors["UNK"])



In [2]:
class OptionParser:
    def __init__(self, train, train_file_type, external_embedding, external_embedding_type, wembedding_dims, lstm_dims):
        self.train = train
        self.train_file_type = train_file_type
        self.external_embedding = external_embedding
        self.external_embedding_type = external_embedding_type
        self.wembedding_dims = wembedding_dims
        self.lstm_dims = lstm_dims
    
options = OptionParser("/home/huseyin/Desktop/Security/data/whyper/Read_Calendar.xls",
                       "excel",
                       "/home/huseyin/Desktop/Security/data/subset_fasttext.bin",
                       "pickle",
                       300,
                       128)


ext_embeddings_arg, ext_emb_dim_arg = Utils.load_embeddings_file(options.external_embedding, options.external_embedding_type)

In [3]:
import dynet_config
# Declare GPU as the default device type
dynet_config.set_gpu()
# Set some parameters manualy
dynet_config.set(mem=400,random_seed=9)
# Initialize dynet import using above configuration in the current scope

import dynet as dy

from numpy import inf
import random
random.seed(33)

class SimpleModel:
    def __init__(self, vocab, w2i, permissions, options):
        self.model = dy.ParameterCollection()
        self.trainer = dy.AdamTrainer(self.model)
        
        self.w2i = w2i
        self.i2w = {w2i[w]:w for w in w2i}
        self.wdims = options.wembedding_dims
        self.ldims = options.lstm_dims
        self.all_permissions = permissions
        self.train_file_type = options.train_file_type
        
        self.wlookup = self.model.add_lookup_parameters((len(w2i), self.wdims)) #PAD, and INITIAL tokens?
        if options.external_embedding is not None:
            ext_embeddings, ext_emb_dim =  Utils.load_embeddings_file(options.external_embedding, options.external_embedding_type)
            assert (ext_emb_dim == self.wdims)
            print("Initializing word embeddings by pre-trained vectors")
            count = 0
            for word in self.w2i:
                if word in ext_embeddings:
                    count += 1
                    self.wlookup.init_row(self.w2i[word], ext_embeddings[word])
            self.ext_embeddings = ext_embeddings
            print("Vocab size: %d; #words having pretrained vectors: %d" % (len(self.w2i), count))
            
        self.sentence_rnn = [dy.SimpleRNNBuilder(1, self.wdims, self.ldims, self.model)] # Try bi-rnn and lstm
        self.permission_rrn = [dy.SimpleRNNBuilder(1, self.wdims, self.ldims, self.model)] # Try bi-rnn and lstm
    
    def cos_similiariy(self, v1, v2):
        from numpy import dot
        from numpy.linalg import norm
        return dot(v1, v2)/(norm(v1)*norm(v2))
    
    def cosine_proximity(self, pred, gold):
        def l2_normalize(x):
            square_sum = dy.sqrt(dy.bmax(dy.sum_elems(dy.square(x)), np.finfo(float).eps * dy.ones((1))[0]))
            return dy.cdiv(x, square_sum)

        y_true = l2_normalize(pred)
        y_pred = l2_normalize(gold)
        return -dy.sum_elems(dy.cmult(y_true, y_pred))
    
    def cosine_loss(self, pred, gold):
        sn1 = dy.l2_norm(pred)
        sn2 = dy.l2_norm(gold)
        mult = dy.cmult(sn1,sn2)
        dot = dy.dot_product(pred,gold)
        div = dy.cdiv(dot,mult)
        y = dy.scalarInput(2)
        res = dy.cdiv(1-div,y)
        return res
    
    def sentence_permission_sim(self, sentences, perm):
        max_sim = -inf
        max_index = 0
        for index, sentence_enc in enumerate(sentences):
            sim = self.cos_similiariy(sentence_enc, perm)
            if max_sim < sim: 
                max_sim = sim
                max_index = index
        return max_sim, max_index

    def statistics(self, similarities):
        statistics = {}
        for app_id in similarities.keys():
            statistics[app_id] = {"related": { "all" : []},
                                  "unrelated": {"all" : []}}
            max_related, max_unrelated = -inf, -inf
            avg_related, avg_unrelated = 0, 0
            for related_p in similarities[app_id]["related"]:
                statistics[app_id]["related"]["all"].append(related_p[1])

            for unrelated_p in similarities[app_id]["unrelated"]:
                statistics[app_id]["unrelated"]["all"].append(unrelated_p[1])
        return statistics
    
    @logging
    def train(self, documents):
        tagged_loss = 0
        untagged_loss = 0 
        for doc in documents:
            if doc.description:                
                #Sentence encoding
                sentence_enc_s = []
                for sentence,tag in zip(doc.description, doc.tags):
                    sentence_enc_s.append([])
                    if tag == 1 or tag == 2 or tag == 3:
                        for window in sentence:
                            permission_vecs = {}
                            # gather all permission encoding of permissions
                            for perm  in self.all_permissions:
                                rnn_forward = self.permission_rrn[0].initial_state()
                                for entry in perm.pphrase:
                                    vec = self.wlookup[int(self.w2i.get(entry, 0))]
                                    rnn_forward = rnn_forward.add_input(vec)
                                permission_vecs[perm.ptype] = rnn_forward.output()
                                
                            rnn_forward = self.sentence_rnn[0].initial_state()
                            for entry in window:
                                vec = self.wlookup[int(self.w2i.get(entry, 0))]
                                rnn_forward = rnn_forward.add_input(vec)
                            loss = []
                            total_e = 0
                            for perm in self.all_permissions:
                                if perm in doc.permissions:
                                    e = self.cosine_loss(rnn_forward.output(), permission_vecs[perm.ptype])
                                    loss.append(1-e)
                                else:
                                    e = self.cosine_loss(rnn_forward.output(), permission_vecs[perm.ptype])
                                    loss.append(e)
                            loss = dy.esum(loss)
                            tagged_loss += loss.scalar_value()
                            loss.backward()
                            self.trainer.update()
                            dy.renew_cg()
                    
                    elif tag == 0:
                        for window in sentence:
                            permission_vecs = {}
                            # gather all permission encoding of permissions
                            for perm  in self.all_permissions:
                                rnn_forward = self.permission_rrn[0].initial_state()
                                for entry in perm.pphrase:
                                    vec = self.wlookup[int(self.w2i.get(entry, 0))]
                                    rnn_forward = rnn_forward.add_input(vec)
                                permission_vecs[perm.ptype] = rnn_forward.output()
                                
                            rnn_forward = self.sentence_rnn[0].initial_state()
                            for entry in window:
                                vec = self.wlookup[int(self.w2i.get(entry, 0))]
                                rnn_forward = rnn_forward.add_input(vec)
                            loss = []
                            for perm in doc.permissions:
                                loss.append(self.cosine_loss(rnn_forward.output(), permission_vecs[perm.ptype]))
                            loss = dy.esum(loss)
                            untagged_loss += loss.scalar_value()
                            loss.backward()
                            self.trainer.update()
                            dy.renew_cg()
        print("Total loss : {} - Tagged Loss {} - Untagged loss {}".format(tagged_loss+untagged_loss, tagged_loss, untagged_loss))
                            
    @logging
    def test(self, documents, print_mode=False):
        document_permission_similiarities = {}
        permission_vecs = {}
        # gather all permission encoding of permissions
        for perm  in self.all_permissions:
            rnn_forward = self.permission_rrn[0].initial_state()
            for entry in perm.pphrase:
                vec = self.wlookup[int(self.w2i.get(entry, 0))]
                rnn_forward = rnn_forward.add_input(vec)
            permission_vecs[perm.ptype] = rnn_forward.output().npvalue()
        for doc in documents:
            if doc.description:   
                document_permission_similiarities[doc.id] = {"related": [], "unrelated" : []}
                if print_mode:
                    print("\n\nDocument {}".format(doc.id))
                    for sent_id,sent in enumerate(doc.raw_sentences):
                        print("Sentence ID {} : {}".format(sent_id + 1, sent))

                for sent_id, (sentence, tag) in enumerate(zip(doc.description, doc.tags)):
                    sentence_enc_s = []
                    if tag == 1 or tag == 2 or tag == 3:
                        for window in sentence:
                            rnn_forward = self.sentence_rnn[0].initial_state()
                            for entry in window:
                                vec = self.wlookup[int(self.w2i.get(entry, 0))]
                                rnn_forward = rnn_forward.add_input(vec)
                            if rnn_forward.output() is not None:
                                rnn_forward.output().npvalue()
                                sentence_enc_s.append(rnn_forward.output().npvalue())
                            dy.renew_cg()

                        for perm in self.all_permissions:
                            max_sim, max_index = self.sentence_permission_sim(sentence_enc_s, permission_vecs[perm.ptype])
                            if perm in doc.permissions:
                                document_permission_similiarities[doc.id]["related"].append((perm.ptype, max_sim))
                                if print_mode:
                                    print("Sentence ID {} Dependency {}".format(doc.id, sent_id+1, sentence[max_index]))
                            else:
                                document_permission_similiarities[doc.id]["unrelated"].append((perm.ptype, max_sim))
        return document_permission_similiarities
    
    @logging
    def train_test_split(self, file_path, window_size=2):
        documents = []
        for doc in Utils.get_data(file_path,
                                self.w2i, 
                                sequence_type="windowed", 
                                file_type=self.train_file_type, 
                                window_size=window_size, 
                                lower=True):
            documents.append(doc)
        random.shuffle(documents)
        split_point = (3*len(documents))//4
        return documents[:split_point], documents[split_point:]


In [4]:
%%time
words, w2i, permissions = Utils.vocab(options.train, file_type=options.train_file_type)


CPU times: user 4.36 s, sys: 344 ms, total: 4.7 s
Wall time: 8.13 s


In [5]:
%%time
model = SimpleModel(words, w2i, permissions, options, ext_embeddings_arg, ext_emb_dim_arg)

Initializing word embeddings by pre-trained vectors
Vocab size: 4834; #words having pretrained vectors: 4289
CPU times: user 112 ms, sys: 0 ns, total: 112 ms
Wall time: 111 ms


In [ ]:
@logging
def draw_histogram(data, img_name):
    stats = model.statistics(data)   
    related_all = []
    unrelated_all = []
    for doc_id in stats:
        related_all.extend([i for i in stats[doc_id]["related"]["all"] if i > -inf])
        unrelated_all.extend([i for i in stats[doc_id]["unrelated"]["all"] if i > -inf])
        
    from matplotlib import pyplot

    pyplot.title("All similarity")
    pyplot.hist(related_all, bins='auto', alpha=0.5, label='related')
    pyplot.hist(unrelated_all, bins='auto', alpha=0.5, label='unrelated')
    pyplot.legend(loc='upper right')
    pyplot.savefig(img_name)
    pyplot.clf()

train_data, test_data = model.train_test_split(options.train)
similarities = model.test(test_data)
draw_histogram(similarities, "trained_epoch_{}.png".format(0))
model.test(test_data)

for i in range(10):
    print("Epoch {}".format(i+1))
    model.train(train_data)
    similarities = model.test(test_data)
    draw_histogram(similarities, "trained_epoch_{}.png".format(i+1))




Epoch 1
> <ipython-input-3-d4eb11094994>(138)train()
-> e = self.cosine_loss(rnn_forward.output(), permission_vecs[perm.ptype])


(Pdb)  n


v1  [ 0.28197637  0.26836446 -0.5771116   0.29709083 -0.06676818  0.12649925
 -0.46922001 -0.02415132 -0.11942988  0.06959423  0.39799285  0.07138018
  0.34649134  0.3245213   0.40272585  0.56679249 -0.20529702  0.26524231
 -0.12629075 -0.21554518  0.13492306  0.2466245   0.59949291  0.18071526
 -0.00242077 -0.22546658  0.1052971   0.03577831  0.38247961 -0.1364498
  0.07308594 -0.06833828 -0.17871222  0.25737375 -0.43688616  0.2661911
 -0.00721624 -0.02983778  0.10380994 -0.57350576 -0.12251846 -0.25539702
  0.00701244 -0.21114126 -0.3260105  -0.32533237  0.03239485 -0.04423056
 -0.3075071  -0.10436306 -0.11462587  0.44353887  0.00139291  0.27557957
 -0.28767073  0.32065353 -0.32273692 -0.18010016 -0.09110064  0.20939957
  0.23760918  0.13514437 -0.0539509  -0.08581656 -0.19230121 -0.32899281
  0.07909564 -0.1419896   0.0435673  -0.38266981 -0.00761344  0.44551566
 -0.11568639  0.21034892 -0.33972254  0.16687268 -0.14262739  0.25385362
  0.11944853  0.49373591 -0.32141697 -0.43412757 

(Pdb)  n


0.5592118501663208
> <ipython-input-3-d4eb11094994>(140)train()
-> loss.append(1-e)
